### we need to Download ChromeDriver
We need to download latest stable release of ChromeDriver from:
https://chromedriver.chromium.org/


In [1]:
#### we need to install the following packages
#### pip install selenium, requests, re, json, time, logging, collections, bs4 (BeautifulSoup),       wget

In [1]:
#imports here
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time

### Step 1: Disable Alerts/Notifications



In [2]:

chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

### Step 2: Log into your personal Facebook account
MAKE SURE to replace <b><i>my_username</i></b> and <b><i>my_password</i></b> with your own unique values


In [3]:
#specify the path to chromedriver.exe (download and save on your computer)
driver = webdriver.Chrome('/Users/tulasiramponaganti/Desktop/WebscrapingFacebook-main/chromedriver',chrome_options=chrome_options)

#open the webpage
driver.get("http://www.facebook.com")

<ipython-input-3-89e1e664e78c>:2: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('/Users/tulasiramponaganti/Desktop/WebscrapingFacebook-main/chromedriver',chrome_options=chrome_options)


In [4]:
#target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

#enter username and password
username.clear()
username.send_keys("tulasi*****@gmail.com")
password.clear()
password.send_keys("******")

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

#We are logged in!

### Step 3: Extract All the Photos

In [7]:
#wait 5 seconds to allow your new page to load
time.sleep(5)
images = [] 

#itterate over both uploaded and tagged images respectively
for i in ["photos_by"]:
    
    driver.get("https://www.facebook.com/tulasi.ram.56863/" + i + "/")
    time.sleep(5)
    
    #scroll down
    #increase the range to sroll more
    #example: range(0,10) scrolls down 650+ images
    for j in range(0,5):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(10)

    #target all the link elements on the page
    anchors = driver.find_elements_by_tag_name('a')
    anchors = [a.get_attribute('href') for a in anchors]
    #narrow down all links to image links only
    anchors = [a for a in anchors if str(a).startswith("https://www.facebook.com/photo")]
    
    print('Found ' + str(len(anchors)) + ' links to images')
    
    #extract the [1]st image element in each link
    for a in anchors:
        driver.get(a) #navigate to link
        time.sleep(5) #wait a bit
        img = driver.find_elements_by_tag_name("img")
        images.append(img[1].get_attribute("src")) #may change in future to img[?]

print('I scraped '+ str(len(images)) + ' images!')

Found 57 links to images
I scraped 57 images!


### Make sure you got the right images!

Before we move on, please check if the first URL leads to the first image you wanted to scrape. If not, please addust the index of your <b>img</b> variable from 1 to what's currently relevant (in the previous cell)

In [8]:
 print(anchors)

['https://www.facebook.com/photo/?fbid=3646703205441584&set=a.730857030359564', 'https://www.facebook.com/photo.php?fbid=3646703205441584&set=pb.100003058220430.-2207520000..&type=3', 'https://www.facebook.com/photo.php?fbid=3645307285581176&set=pb.100003058220430.-2207520000..&type=3', 'https://www.facebook.com/photo.php?fbid=3330814027030505&set=pb.100003058220430.-2207520000..&type=3', 'https://www.facebook.com/photo.php?fbid=3221731437938765&set=pb.100003058220430.-2207520000..&type=3', 'https://www.facebook.com/photo.php?fbid=3216131868498722&set=pb.100003058220430.-2207520000..&type=3', 'https://www.facebook.com/photo.php?fbid=3077251942386716&set=pb.100003058220430.-2207520000..&type=3', 'https://www.facebook.com/photo.php?fbid=3047083218736922&set=pb.100003058220430.-2207520000..&type=3', 'https://www.facebook.com/photo.php?fbid=3046881388757105&set=pb.100003058220430.-2207520000..&type=3', 'https://www.facebook.com/photo.php?fbid=3046733832105194&set=pb.100003058220430.-220752

### Step 4: saving all images links into csv 


In [11]:
import pandas as pd
df = pd.DataFrame(anchors) 
    
# saving the dataframe 
#df.to_csv('GFG.csv') 
df.to_csv(r'images.csv', index = False)

In [16]:
#Step 5: If you want Save all the photos in the new directory

#import os
#path = os.getcwd()
#path = os.path.join(path, "SCRAPED_PHOTOS")
#create the directory
#os.mkdir(path)

#import wget
#download images
#counter = 0
#for image in images:
    #save_as = os.path.join(path, str(counter) + '.jpg')
    #wget.download(image, save_as)
    #counter += 1

In [24]:
## Extracting all comments that are linked in with images
import requests
import re
import json
import time
import logging
import pandas
from collections import OrderedDict
from bs4 import BeautifulSoup


url = "https://www.facebook.com/tulasi.ram.56863/"

def extract_comments(session, base_url, post_bs, post_url):
    """Extracts all coments from post
    """
    comments = list()
    show_more_url = post_bs.find('a', href=re.compile('/story\.php\?story'))['href']
    first_comment_page = True

    logging.info('Scraping comments from {}'.format(post_url))
    while True:

        logging.info('[!] Scraping comments.')
        time.sleep(3)
        if first_comment_page:
            first_comment_page = False
        else:
            post_bs = get_bs(session, base_url+show_more_url)
            time.sleep(3)
        
        try:
            comments_elements = post_bs.find('div', id=re.compile('composer')).next_sibling\
                .find_all('div', id=re.compile('^\d+'))
        except Exception:
            pass

        if len(comments_elements) != 0:
            logging.info('[!] There are comments.')
        else:
            break
        
        for comment in comments_elements:
            comment_data = OrderedDict()
            comment_data['text'] = list()
            try:
                comment_strings = comment.find('h3').next_sibling.strings
                for string in comment_strings:
                    comment_data['text'].append(string)
            except Exception:
                pass
            
            try:
                media = comment.find('h3').next_sibling.next_sibling.children
                if media is not None:
                    for element in media:
                        comment_data['media_url'] = element['src']
                else:
                    comment_data['media_url'] = ''
            except Exception:
                pass
            
            
            comment_data['text'] = comment.find('h3').a['href'].split('?')[0]
            comments.append(dict(comment_data))
        
        show_more_url = post_bs.find('a', href=re.compile('/story\.php\?story'))
        if 'View more' in show_more_url.text:
            logging.info('[!] More comments.')
            show_more_url = show_more_url['href']
        else:
            break
    
    return comments

In [ ]:
def save_data(data):
    """Converts data to JSON.
    """
    with open('posts_data.json', 'w') as json_file:
        json.dump(data, json_file, indent=4)

        df1 =  pd.read_json("posts_data.json")
        df1.to_csv("comments.csv")
        

In [ ]:
data = pd.read_csv("comments.csv", encoding='UTF-8')

data1 = pd.merge(left=df1, right=df, left_on='index', right_on='index')

data1.to_excel("final.xlsx")

In [ ]:
## Storing data into mongo db


In [1]:
import csv
import pymongo
import json
import pandas as pd

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017")


In [20]:
df = pd.read_excel("final.xlsx")
df1 = df.to_csv("final.csv")
df.head(50)

,Links,text
0,https://www.facebook.com/photo/?fbid=364670320...,"its awesome, looking good"
1,https://www.facebook.com/photo.php?fbid=364670...,you look like hero
2,https://www.facebook.com/photo.php?fbid=364530...,super ram
3,https://www.facebook.com/photo.php?fbid=333081...,Hi how are you
4,https://www.facebook.com/photo.php?fbid=322173...,"where are you?, how was your family?"
5,https://www.facebook.com/photo.php?fbid=321613...,Are you in india
6,https://www.facebook.com/photo.php?fbid=307725...,amazing pic
7,https://www.facebook.com/photo.php?fbid=304708...,how was your nephew?
8,https://www.facebook.com/photo.php?fbid=304688...,you look great
9,https://www.facebook.com/photo.php?fbid=304673...,tell me when you will be in india


In [21]:
data = df.to_dict(orient ="records")

In [22]:
data

[{'Links': 'https://www.facebook.com/photo/?fbid=3646703205441584&set=a.730857030359564',
  'text ': 'its awesome, looking good '},
 {'Links': 'https://www.facebook.com/photo.php?fbid=3646703205441584&set=pb.100003058220430.-2207520000..&type=3',
  'text ': 'you look like hero'},
 {'Links': 'https://www.facebook.com/photo.php?fbid=3645307285581176&set=pb.100003058220430.-2207520000..&type=3',
  'text ': 'super ram'},
 {'Links': 'https://www.facebook.com/photo.php?fbid=3330814027030505&set=pb.100003058220430.-2207520000..&type=3',
  'text ': 'Hi how are you'},
 {'Links': 'https://www.facebook.com/photo.php?fbid=3221731437938765&set=pb.100003058220430.-2207520000..&type=3',
  'text ': 'where are you?, how was your family?'},
 {'Links': 'https://www.facebook.com/photo.php?fbid=3216131868498722&set=pb.100003058220430.-2207520000..&type=3',
  'text ': 'Are you in india'},
 {'Links': 'https://www.facebook.com/photo.php?fbid=3077251942386716&set=pb.100003058220430.-2207520000..&type=3',
  'te

In [23]:
db = client["MachineLearning"]

In [24]:
print(db)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'MachineLearning')


In [26]:
db.sample_training.insert_many(data)

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 60ecc61b4463c0a2b584009b, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused')>]>

In [ ]:


df = pd.read_csv("final.csv")

data = df.to_dict(orient ="records")
db = client["MachineLearning"]

print(db)

db.MachineLearning.insert_many(data)